In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.metrics as metrics 

In [2]:
# !pip install nltk

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
listings_summary = pd.read_csv("listing_summary.csv")
listings_summary.head()

,id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,...,review_scores_value,requires_license,license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,It is located in the former East Berlin area o...,"This is my home, not a hotel. I rent out occas...","Close to U-Bahn U8 and U2 (metro), Trams M12, ...","Simple kitchen/cooking, refrigerator, microwav...",Always available,...,9.0,t,NaN,f,f,strict_14_with_grace_period,f,f,4,3.76
1,2695,Prenzlauer Berg close to Mauerpark,NaN,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,NaN,NaN,Within walking distance you'll find the S-Bahn...,Außer deinem Zimmer kannst du noch die Küche u...,NaN,...,10.0,t,NaN,f,f,flexible,f,f,1,1.42
2,3176,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor (68m2) apartment on Kollwitzplatz/ P...,This beautiful first floor apartment is situa...,The neighbourhood is famous for its variety of...,We welcome FAMILIES and cater especially for y...,"We are 5 min walk away from the tram M2, whic...",The apartment will be entirely yours. We are c...,Feel free to ask any questions prior to bookin...,...,9.0,t,NaN,t,f,strict_14_with_grace_period,f,f,1,1.25
3,3309,BerlinSpot Schöneberg near KaDeWe,First of all: I prefer short-notice bookings. ...,"Your room is really big and has 26 sqm, is ver...",First of all: I prefer short-notice bookings. ...,"My flat is in the middle of West-Berlin, direc...",The flat is a strictly non-smoking facility! A...,The public transportation is excellent: Severa...,I do have a strictly non-smoker-flat. Keep th...,I'm working as a freelancing photographer. My ...,...,9.0,t,NaN,f,f,strict_14_with_grace_period,f,f,1,0.39
4,7071,BrightRoom with sunny greenview!,Cozy and large room in the beautiful district ...,"The BrightRoom is an approx. 20 sqm (215ft²), ...",Cozy and large room in the beautiful district ...,"Great neighborhood with plenty of Cafés, Baker...",I hope you enjoy your stay to the fullest! Ple...,Best access to other parts of the city via pub...,"The guests have access to the bathroom, a smal...",I am glad if I can give you advice or help as ...,...,9.0,t,NaN,f,f,moderate,f,f,1,1.75


In [28]:
listings_summary.columns

Index(['id', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'picture_url', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_location', 'host_about', 'host_response_rate',
       'host_response_time', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'latitude', 'longitude', 'is_location_exact',
       'property_type', 'room_type', 'accommodates', 'bedrooms', 'bathrooms',
       'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price',
       'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_inc

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/leewaxman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leewaxman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leewaxman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
from collections import Counter
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

In [16]:
!pip install itertools

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [10]:
n = listings_summary.shape[0]
listings_summary["new_summary"] = pd.Series() 

<ipython-input-10-c13438b63876>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  listings_summary["new_summary"] = pd.Series()


In [56]:
for i in range(0,n):
    if listings_summary.loc[i,'summary'] == listings_summary.loc[i,'description']:
        listings_summary.loc[i, "new_summary"] = listings_summary.loc[i,'summary']
    elif isnan(listings_summary.loc[i,'summary']):
        listings_summary.loc[i, "new_summary"] = listings_summary.loc[i,'summary']
    else:
        listings_summary.loc[i, "new_summary"] = listings_summary.loc[i,'description']
        

In [9]:
def isnan(value):
    try:
        import math
        math.isnan(float(value))
        return False
    except:
        return True

# Summary and description analysis

In [75]:

################
### Data Cleansing
################
wrds = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.index[i]
    if (isnan(listings_summary.loc[i,'new_summary'])):
        ### Preprocessing
        text = listings_summary.loc[i,'new_summary'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds.append(bowlst)
        dftext[id] = dict(bow)

In [76]:
import itertools
wrds_joined = list(itertools.chain.from_iterable(wrds))
print('total: ',len(wrds_joined))
print('unique: ',len(set(wrds_joined)))

total:  770621
unique:  35189


In [77]:
### get the unique words as base
wrds_unique = list(set(wrds_joined))

In [78]:
text_ff = pd.DataFrame(dftext)
text_ff.head(20)

,0,1,2,3,4,5,6,7,8,9,...,22541,22543,22544,22545,22546,22547,22548,22549,22550,22551
great,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
location,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
,13.0,14.0,3.0,12.0,1.0,10.0,11.0,22.0,1.0,8.0,...,8.0,9.0,13.0,1.0,16.0,2.0,10.0,12.0,9.0,16.0
30,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sq,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
meters,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wood,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
flooredhigh,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ceiling,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
text_ff = text_ff.transpose()
text_ff.head()

,great,location,,30,75,sq,meters,wood,flooredhigh,ceiling,...,rudergerät,notebook,undivided,unserere,mehrgenerationenhaus,ausschliesslich,kennenlerne,lebensnotwendigen,casper,multilanguage
0,1.0,1.0,13.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,14.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
text_ff = text_ff.fillna(0)
text_ff.head()

,great,location,,30,75,sq,meters,wood,flooredhigh,ceiling,...,rudergerät,notebook,undivided,unserere,mehrgenerationenhaus,ausschliesslich,kennenlerne,lebensnotwendigen,casper,multilanguage
0,1.0,1.0,13.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,14.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
text_ff = text_ff.transpose()
text_ff

,0,1,2,3,4,5,6,7,8,9,...,22541,22543,22544,22545,22546,22547,22548,22549,22550,22551
great,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
location,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
,13.0,14.0,3.0,12.0,1.0,10.0,11.0,22.0,1.0,8.0,...,8.0,9.0,13.0,1.0,16.0,2.0,10.0,12.0,9.0,16.0
30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ausschliesslich,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kennenlerne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lebensnotwendigen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
casper,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [84]:
text_ff["sum"] = text_ff.sum(axis=1)


,0,1,2,3,4,5,6,7,8,9,...,22543,22544,22545,22546,22547,22548,22549,22550,22551,sum
great,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1736.0
location,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1548.0
,13.0,14.0,3.0,12.0,1.0,10.0,11.0,22.0,1.0,8.0,...,9.0,13.0,1.0,16.0,2.0,10.0,12.0,9.0,16.0,214357.0
30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,395.0
75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ausschliesslich,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
kennenlerne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
lebensnotwendigen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
casper,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [88]:
text_ff = text_ff.sort_values(by=["sum"])
text_ff.tail(30)

,0,1,2,3,4,5,6,7,8,9,...,22543,22544,22545,22546,22547,22548,22549,22550,22551,sum
sum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
multilanguage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
150x210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
gartenwohnung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
northwestern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
wgzimmerchen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
sunnyfully,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
untergebrachten,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
bard,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
selbstverständlcih,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
import nltk
from nltk.stem import SnowballStemmer

In [11]:
nltk.download('crubadan')

[nltk_data] Downloading package crubadan to
[nltk_data]     /Users/leewaxman/nltk_data...
[nltk_data]   Package crubadan is already up-to-date!


True

In [12]:
n = listings_summary.shape[0]
lang_space = []
tc = nltk.classify.textcat.TextCat() 
for i in range(0,n):
    if (isnan(listings_summary.loc[i,'space'])):
        lang = tc.guess_language(listings_summary.loc[i,'space'])
        lang_space.append(lang)

space = pd.DataFrame(listings_summary["space"], lang_space)
space

KeyboardInterrupt: 

# space

In [21]:

################
### Data Cleansing
################
wrds_space = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.loc[i,'id']
    if (isnan(listings_summary.loc[i,'space'])):
        ### Preprocessing
        text = listings_summary.loc[i,'space'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds_space.append(bowlst)
        dftext[id] = dict(bow)

In [22]:
import itertools
wrds_joined_space = list(itertools.chain.from_iterable(wrds_space))
print('total: ',len(wrds_joined_space))
print('unique: ',len(set(wrds_joined_space)))

total:  594945
unique:  33278


In [23]:
### get the unique words as base
wrds_unique_space = list(set(wrds_joined_space))

In [24]:
text_ff_space = pd.DataFrame(dftext)
text_ff_space.head(20)

,2015,2695,3176,3309,7071,9991,14325,16401,16644,17409,...,29830140,29830381,29848527,29848907,29849186,29849958,29851223,29856646,29857108,29866805
location,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
,40.0,14.0,24.0,35.0,23.0,51.0,27.0,22.0,33.0,28.0,...,1.0,38.0,25.0,11.0,11.0,11.0,2.0,22.0,3.0,8.0
„,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
einliegerwohnung,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
“,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
extention,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
larger,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
apartment,3.0,2.0,2.0,NaN,1.0,NaN,3.0,1.0,1.0,NaN,...,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
separate,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
entrance,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
text_ff_space = text_ff_space.transpose()
text_ff_space = text_ff_space.fillna(0)
text_ff_space.head()

,location,,„,einliegerwohnung,“,extention,larger,apartment,separate,entrance,...,siegfriesdshöfen,siegfried,farms,hackeschenhöfe,siegfriedshöfe,terrain,energieverbrauchskennwert,kwh,couchsesselbereich,wachmaschine
2015,1.0,40.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2695,0.0,14.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3176,0.0,24.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3309,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7071,1.0,23.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# neighborhood overview


In [29]:

################
### Data Cleansing
################
wrds_negihborhood_overview = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.loc[i,'id']
    if (isnan(listings_summary.loc[i,'neighborhood_overview'])):
        ### Preprocessing
        text = listings_summary.loc[i,'neighborhood_overview'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds_negihborhood_overview.append(bowlst)
        dftext[id] = dict(bow)

In [30]:
import itertools
wrds_joined_negihborhood_overview = list(itertools.chain.from_iterable(wrds_negihborhood_overview))
print('total: ',len(wrds_joined_negihborhood_overview))
print('unique: ',len(set(wrds_joined_negihborhood_overview)))

total:  371268
unique:  24562


In [31]:
### get the unique words as base
wrds_unique_space = list(set(wrds_joined_space))

In [32]:
text_ff_space = pd.DataFrame(dftext)
text_ff_space.head(20)

,2015,3176,3309,7071,9991,16644,17409,17904,20858,21869,...,29830204,29846605,29848907,29849186,29849958,29851223,29856646,29856708,29866805,29867352
located,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
former,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
east,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
berlin,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN
area,2.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kastanienallee,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zionskirchplatz,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
,21.0,8.0,11.0,19.0,7.0,6.0,8.0,12.0,5.0,6.0,...,11.0,2.0,1.0,1.0,1.0,6.0,12.0,3.0,8.0,2.0
berlins,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
natural,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
text_ff_space = text_ff_space.transpose()
text_ff_space = text_ff_space.fillna(0)
text_ff_space.head()

,located,former,east,berlin,area,kastanienallee,zionskirchplatz,,berlins,natural,...,100meter,günztlerstrasse,berlinerstrasse,domicile,framework,academic,muttistyle,entzückt,unnachahmlichen,tagesordnung
2015,1.0,1.0,1.0,1.0,2.0,1.0,1.0,21.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3309,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7071,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# notes

In [34]:

################
### Data Cleansing
################
wrds_notes = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.loc[i,'id']
    if (isnan(listings_summary.loc[i,'notes'])):
        ### Preprocessing
        text = listings_summary.loc[i,'notes'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds_notes.append(bowlst)
        dftext[id] = dict(bow)

In [35]:
wrds_joined_notes = list(itertools.chain.from_iterable(wrds_notes))
print('total: ',len(wrds_joined_notes))
print('unique: ',len(set(wrds_joined_notes)))

total:  157217
unique:  16591


In [37]:
wrds_unique_note = list(set(wrds_joined_notes))

In [38]:
text_ff_note = pd.DataFrame(dftext)
text_ff_note.head(20)

,2015,3176,3309,7071,20858,22415,22677,26543,28268,29279,...,29823132,29827481,29829280,29830140,29848527,29848907,29849186,29849958,29856646,29866805
home,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
,3.0,12.0,4.0,2.0,11.0,44.0,19.0,27.0,13.0,4.0,...,1.0,5.0,3.0,1.0,18.0,8.0,8.0,8.0,2.0,2.0
hotel,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rent,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
occasionally,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stay,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
friend,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
welcome,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
families,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cater,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
text_ff_note = text_ff_note.transpose()
text_ff_note = text_ff_note.fillna(0)
text_ff_note.head()

,home,,hotel,rent,occasionally,stay,friend,welcome,families,cater,...,goaloriented,schriftlicher,zeitmiete,225,reinigungsdienst,vermittelt,filmproduktionen,mittlere,10122018,18012018
2015,1.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3176,0.0,12.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3309,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7071,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20858,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# transit

In [40]:

################
### Data Cleansing
################
wrds_transit = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.loc[i,'id']
    if (isnan(listings_summary.loc[i,'transit'])):
        ### Preprocessing
        text = listings_summary.loc[i,'transit'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds_transit.append(bowlst)
        dftext[id] = dict(bow)


In [41]:
wrds_joined_transit = list(itertools.chain.from_iterable(wrds_transit))
print('total: ',len(wrds_joined_transit))
print('unique: ',len(set(wrds_joined_transit)))

total:  299204
unique:  14330


In [42]:
wrds_unique_transit = list(set(wrds_joined_transit))


In [43]:
text_ff_transit = pd.DataFrame(dftext)
text_ff_transit.head(20)

,2015,2695,3176,3309,7071,9991,16644,17409,17904,20858,...,29824905,29825372,29830204,29848907,29849186,29849958,29856646,29856708,29866805,29867352
close,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
ubahn,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u8,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
u2,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
,9.0,5.0,5.0,10.0,4.0,7.0,1.0,3.0,8.0,4.0,...,4.0,7.0,8.0,2.0,2.0,2.0,4.0,1.0,6.0,1.0
metro,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trams,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
m12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
m10,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
m8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
text_ff_transit = text_ff_transit.transpose()
text_ff_transit = text_ff_transit.fillna(0)
text_ff_transit.head()

,close,ubahn,u8,u2,,metro,trams,m12,m10,m8,...,ebenalls,rip,wollank,facilites,fernreisebahnhof,gehminunten,erhohlungspark,kindergartens,schools,anwohnergebiet
2015,2.0,1.0,2.0,1.0,9.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2695,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3176,0.0,0.0,0.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3309,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7071,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Access

In [49]:

################
### Data Cleansing
################
wrds_access = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.loc[i,'id']
    if (isnan(listings_summary.loc[i,'access'])):
        ### Preprocessing
        text = listings_summary.loc[i,'access'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds_access.append(bowlst)
        dftext[id] = dict(bow)



In [50]:
wrds_joined_access = list(itertools.chain.from_iterable(wrds_access))
print('total: ',len(wrds_joined_access))
print('unique: ',len(set(wrds_joined_access)))


total:  166144
unique:  12702


In [51]:
wrds_unique_access = list(set(wrds_joined_access))


In [52]:
text_ff_access = pd.DataFrame(dftext)
text_ff_access.head(20)

,2015,2695,3176,3309,7071,9991,16644,17409,17904,20858,...,29816356,29816679,29817070,29823132,29824905,29829280,29830381,29838765,29848527,29866805
simple,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kitchencooking,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
,16.0,5.0,3.0,12.0,5.0,1.0,2.0,4.0,9.0,1.0,...,7.0,2.0,2.0,NaN,2.0,1.0,1.0,2.0,1.0,4.0
refrigerator,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
microwave,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
oven,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stove,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dishwasher,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
espresso,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
coffee,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [53]:
text_ff_access = text_ff_access.transpose()
text_ff_access = text_ff_access.fillna(0)
text_ff_access.head()

,simple,kitchencooking,,refrigerator,microwave,oven,stove,dishwasher,espresso,coffee,...,kaffeemühle,kochtöpfe,plattenspieler,iphoneipad,wohnzimmerecke,90cm,internetprovider,workingspace,bullshit,roomdining
2015,1.0,1.0,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2695,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3176,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3309,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7071,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# interaction

In [54]:

################
### Data Cleansing
################
wrds_interaction = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.loc[i,'id']
    if (isnan(listings_summary.loc[i,'interaction'])):
        ### Preprocessing
        text = listings_summary.loc[i,'interaction'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds_interaction.append(bowlst)
        dftext[id] = dict(bow)


In [55]:
wrds_joined_interaction = list(itertools.chain.from_iterable(wrds_interaction))
print('total: ',len(wrds_joined_interaction))
print('unique: ',len(set(wrds_joined_interaction)))


total:  155306
unique:  10413


In [56]:
wrds_unique_interaction = list(set(wrds_joined_interaction))

In [57]:
text_ff_interaction = pd.DataFrame(dftext)
text_ff_interaction.head(20)

,2015,3176,3309,7071,9991,16644,17409,17904,20858,21869,...,29824905,29825372,29827481,29829280,29830140,29830204,29838765,29851223,29856646,29866805
always,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
available,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
feel,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
free,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
ask,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
questions,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
prior,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
booking,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stay,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
,NaN,6.0,28.0,3.0,1.0,2.0,1.0,5.0,13.0,2.0,...,9.0,1.0,5.0,NaN,2.0,NaN,1.0,3.0,4.0,1.0


In [58]:
text_ff_interaction = text_ff_interaction.transpose()
text_ff_interaction = text_ff_interaction.fillna(0)
text_ff_interaction.head()

,always,available,feel,free,ask,questions,prior,booking,stay,,...,vocês,terão,toda,privacidade,em,ambiente,aconchegante,peru,ukrainian,seconds
2015,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3176,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# house_rules

In [59]:

################
### Data Cleansing
################
wrds_house_rules = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.loc[i,'id']
    if (isnan(listings_summary.loc[i,'house_rules'])):
        ### Preprocessing
        text = listings_summary.loc[i,'house_rules'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds_house_rules.append(bowlst)
        dftext[id] = dict(bow)


In [60]:
wrds_joined_house_rules = list(itertools.chain.from_iterable(wrds_house_rules))
print('total: ',len(wrds_joined_house_rules))
print('unique: ',len(set(wrds_joined_house_rules)))

total:  253385
unique:  16526


In [61]:
wrds_unique_house_rules = list(set(wrds_joined_house_rules))

In [62]:
text_ff_house_rules = pd.DataFrame(dftext)
text_ff_house_rules.head(20)

,2015,3176,3309,7071,14325,16644,17409,17904,20858,21869,...,29816679,29817070,29824905,29825691,29830140,29830204,29830381,29848527,29856646,29867352
parties,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
events,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pets,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
smoking,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN
,3.0,6.0,29.0,18.0,2.0,10.0,3.0,9.0,2.0,10.0,...,8.0,1.0,6.0,NaN,4.0,1.0,13.0,18.0,2.0,1.0
even,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
courtyardgarden,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pull,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
door,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shut,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
text_ff_house_rules = text_ff_house_rules.transpose()
text_ff_house_rules = text_ff_house_rules.fillna(0)
text_ff_house_rules.head()

,parties,events,pets,smoking,,even,courtyardgarden,pull,door,shut,...,recyclable,plantfriendly,caro,möblierte,zeitmiete,225,030,reinigungsdienst,vermittelt,filmproduktionen
2015,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3176,1.0,0.0,1.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3309,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7071,1.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14325,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# host_about

In [64]:

################
### Data Cleansing
################
wrds_host_about = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.loc[i,'id']
    if (isnan(listings_summary.loc[i,'host_about'])):
        ### Preprocessing
        text = listings_summary.loc[i,'host_about'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds_host_about.append(bowlst)
        dftext[id] = dict(bow)

In [65]:
wrds_joined_host_about = list(itertools.chain.from_iterable(wrds_host_about))
print('total: ',len(wrds_joined_host_about))
print('unique: ',len(set(wrds_joined_host_about)))

total:  297404
unique:  23635


In [66]:
wrds_unique_host_about = list(set(wrds_joined_host_about))


In [67]:
text_ff_host_about = pd.DataFrame(dftext)
text_ff_host_about.head(20)

,2015,2695,3176,3309,7071,9991,14325,16401,16644,17409,...,29830381,29838765,29842909,29848269,29848527,29849565,29850644,29851223,29857108,29866805
believe,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sharing,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
economy,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
,1.0,7.0,2.0,13.0,15.0,9.0,3.0,5.0,3.0,3.0,...,1.0,9.0,1.0,NaN,1.0,3.0,2.0,7.0,3.0,10.0
living,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wife,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
berlin,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,1.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0
like,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
travelling,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hiking,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
text_ff_host_about = text_ff_host_about.transpose()
text_ff_host_about = text_ff_host_about.fillna(0)
text_ff_host_about.head()

,believe,sharing,economy,,living,wife,berlin,like,travelling,hiking,...,guro,gewand,tüp,aziza,tashkent,uzbekistan,relabel,justizvollzugsanstalt,triathlet,augehmöglichkeiten
2015,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2695,0.0,0.0,0.0,7.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3176,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3309,0.0,0.0,0.0,13.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7071,0.0,0.0,0.0,15.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# room_type

In [69]:

################
### Data Cleansing
################
wrds_room_type = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.loc[i,'id']
    if (isnan(listings_summary.loc[i,'room_type'])):
        ### Preprocessing
        text = listings_summary.loc[i,'room_type'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds_room_type.append(bowlst)
        dftext[id] = dict(bow)


In [70]:
wrds_joined_room_type = list(itertools.chain.from_iterable(wrds_room_type))
print('total: ',len(wrds_joined_room_type))
print('unique: ',len(set(wrds_joined_room_type)))

total:  45104
unique:  5


In [71]:
wrds_unique_room_type = list(set(wrds_joined_room_type))

In [72]:
text_ff_room_type = pd.DataFrame(dftext)
text_ff_room_type.head(20)

,2015,2695,3176,3309,7071,9991,14325,16401,16644,17409,...,29851651,29853463,29856029,29856359,29856646,29856708,29857108,29864272,29866805,29867352
entire,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,...,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN
homeapt,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,...,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN
private,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,1.0,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0
room,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,1.0,...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,1.0,1.0
shared,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [73]:
text_ff_room_type = text_ff_room_type.transpose()
text_ff_room_type = text_ff_room_type.fillna(0)
text_ff_room_type.head()

,entire,homeapt,private,room,shared
2015,1.0,1.0,0.0,0.0,0.0
2695,0.0,0.0,1.0,1.0,0.0
3176,1.0,1.0,0.0,0.0,0.0
3309,0.0,0.0,1.0,1.0,0.0
7071,0.0,0.0,1.0,1.0,0.0


In [87]:
import csv

# open the file in the write mode
with open('room_type.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(text_ff_room_type)

In [115]:
text_ff_room_type["id"] = text_ff_room_type.index

In [116]:
text_ff_room_type.to_csv("room_type.csv", encoding='utf-8', index=False)

# amenities

In [21]:

################
### Data Cleansing
################
wrds_amenities_1 = []
n = listings_summary.shape[0]
dftext = {}
# listings_summary = listings_summary.reset_index()

for i in range(0,n):
    id = listings_summary.loc[i,'id']
    if (isnan(listings_summary.loc[i,'host_verifications'])):
        ### Preprocessing
        text = listings_summary.loc[i,'host_verifications'].lower()
        ### tokenizing
        tokenized_word = word_tokenize(text)
        ### remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokenized_word]
        ### remove stopwords
        stop_words=set(stopwords.words("english"))
        filtered_sent=[]
        for w in stripped:
            if w not in stop_words:
                filtered_sent.append(w)
        ### Lemmatization¶
        lem = WordNetLemmatizer()
    #     lem.lemmatize(filtered_sent,"v")
        lem_strings=[]
        for w in filtered_sent:
            lem.lemmatize(w,"v")
            lem_strings.append(w)
        ### stemming
#         stem = PorterStemmer()
    #     stem.stem(filtered_sent)
        bow = Counter(lem_strings)
        bow.most_common(20)
        bowlst = list(bow.keys())
        wrds_amenities_1.append(bowlst)
        dftext[id] = dict(bow)

In [23]:
import itertools
wrds_joined_amenities = list(itertools.chain.from_iterable(wrds_amenities_1))
print('total: ',len(wrds_joined_amenities))
print('unique: ',len(set(wrds_joined_amenities)))

total:  133412
unique:  22


In [24]:
wrds_unique_amenities = list(set(wrds_joined_amenities))

In [25]:
text_ff_amenities = pd.DataFrame(dftext)
text_ff_amenities.head(20)

,2015,2695,3176,3309,7071,9991,14325,16401,16644,17409,...,29851651,29853463,29856029,29856359,29856646,29856708,29857108,29864272,29866805,29867352
,13.0,11.0,15.0,13.0,11.0,13.0,17.0,13.0,11.0,11.0,...,11.0,17.0,5.0,5.0,13.0,7.0,11.0,11.0,7.0,5.0
email,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
phone,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
reviews,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,...,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN
jumio,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,...,1.0,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN
offlinegovernmentid,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
governmentid,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,...,1.0,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN
facebook,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,...,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN
manualoffline,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
workemail,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
text_ff_amenities = text_ff_amenities.transpose()
text_ff_amenities = text_ff_amenities.fillna(0)
text_ff_amenities.head()

,,email,phone,reviews,jumio,offlinegovernmentid,governmentid,facebook,manualoffline,workemail,...,kba,sentid,google,manualonline,none,zhimaselfie,weibo,sesame,sesameoffline,photographer
2015,13.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2695,11.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3176,15.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3309,13.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7071,11.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
# open the file in the write mode
with open('amenities.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(text_ff_room_type)

In [30]:
text_ff_amenities["id"] = text_ff_amenities.index

In [31]:
text_ff_amenities.to_csv("host_verifications.csv", encoding='utf-8', index=False)

In [100]:
listings_summary.columns

Index(['id', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'picture_url', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_location', 'host_about', 'host_response_rate',
       'host_response_time', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'latitude', 'longitude', 'is_location_exact',
       'property_type', 'room_type', 'accommodates', 'bedrooms', 'bathrooms',
       'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price',
       'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_inc

In [102]:
listings_summary = listings_summary.drop("summary", axis = 1)
listings_summary = listings_summary.drop("space", axis = 1)
listings_summary = listings_summary.drop("description", axis = 1)
listings_summary = listings_summary.drop("neighborhood_overview", axis = 1)
listings_summary = listings_summary.drop("notes", axis = 1)
listings_summary = listings_summary.drop("transit", axis = 1)
listings_summary = listings_summary.drop("access", axis = 1)
listings_summary = listings_summary.drop("interaction", axis = 1)
listings_summary = listings_summary.drop("house_rules", axis = 1)
listings_summary = listings_summary.drop("host_about", axis = 1)
listings_summary = listings_summary.drop("room_type", axis = 1)
listings_summary = listings_summary.drop("amenities", axis = 1)
listings_summary.columns

Index(['id', 'name', 'picture_url', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_location', 'host_response_rate',
       'host_response_time', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'latitude', 'longitude', 'is_location_exact',
       'property_type', 'accommodates', 'bedrooms', 'bathrooms', 'beds',
       'bed_type', 'square_feet', 'price', 'weekly_price', 'monthly_price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'calendar_updated',
       'availability_30', 'availability_60', 'availability_90',
       'availability_36

In [105]:
for index, row in listings_summary.iterrows():
    if isnan(row["host_picture_url"]):
        listings_summary["is_host_picture_url"] = 1
    else:
        listings_summary["is_host_picture_url"] = 0
listings_summary

,id,name,picture_url,host_id,host_url,host_name,host_since,host_location,host_response_rate,host_response_time,...,requires_license,license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,is_host_picture_url
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,https://a0.muscache.com/im/pictures/260fd609-7...,2217,https://www.airbnb.com/users/show/2217,Ian,2008-08-18,"Key Biscayne, Florida, United States",96%,within an hour,...,t,NaN,f,f,strict_14_with_grace_period,f,f,4,3.76,1
1,2695,Prenzlauer Berg close to Mauerpark,https://a0.muscache.com/im/pictures/97d4f6e8-1...,2986,https://www.airbnb.com/users/show/2986,Michael,2008-09-16,"Berlin, Berlin, Germany",NaN,NaN,...,t,NaN,f,f,flexible,f,f,1,1.42,1
2,3176,Fabulous Flat in great Location,https://a0.muscache.com/im/pictures/243355/84a...,3718,https://www.airbnb.com/users/show/3718,Britta,2008-10-19,"Coledale, New South Wales, Australia",100%,within a day,...,t,NaN,t,f,strict_14_with_grace_period,f,f,1,1.25,1
3,3309,BerlinSpot Schöneberg near KaDeWe,https://a0.muscache.com/im/pictures/29054294/b...,4108,https://www.airbnb.com/users/show/4108,Jana,2008-11-07,"Berlin, Berlin, Germany",NaN,NaN,...,t,NaN,f,f,strict_14_with_grace_period,f,f,1,0.39,1
4,7071,BrightRoom with sunny greenview!,https://a0.muscache.com/im/pictures/21278/32a1...,17391,https://www.airbnb.com/users/show/17391,Bright,2009-05-16,"Berlin, Berlin, Germany",100%,within an hour,...,t,NaN,f,f,moderate,f,f,1,1.75,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22547,29856708,Cozy Apartment right in the center of Berlin,https://a0.muscache.com/im/pictures/43de229c-e...,87555909,https://www.airbnb.com/users/show/87555909,Ulisses,2016-08-02,"Berlin, Berlin, Germany",NaN,NaN,...,t,NaN,f,f,flexible,f,f,1,NaN,1
22548,29857108,Altbau/ Schöneberger Kiez / Schlafsofa,https://a0.muscache.com/im/pictures/d2b27398-b...,67537363,https://www.airbnb.com/users/show/67537363,Jörg,2016-04-17,"Berlin, Berlin, Germany",88%,within an hour,...,t,NaN,t,f,flexible,f,f,6,NaN,1
22549,29864272,Artists loft with garden in the center of Berlin,https://a0.muscache.com/im/pictures/0e40a7c4-b...,3146923,https://www.airbnb.com/users/show/3146923,Martin,2012-08-03,"Berlin, Berlin, Germany",NaN,NaN,...,t,NaN,f,f,flexible,f,f,2,NaN,1
22550,29866805,Room for two with private shower / WC,https://a0.muscache.com/im/pictures/5a110f66-e...,36961901,https://www.airbnb.com/users/show/36961901,Arte Luise,2015-06-28,"Berlin, Berlin, Germany",90%,within a few hours,...,t,NaN,f,f,flexible,f,f,3,NaN,1


In [106]:
for index, row in listings_summary.iterrows():
    if isnan(row["host_url"]):
        listings_summary["is_host_url"] = 1
    else:
        listings_summary["is_host_url"] = 0
listings_summary.head()

,id,name,picture_url,host_id,host_url,host_name,host_since,host_location,host_response_rate,host_response_time,...,license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,is_host_picture_url,is_host_url
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,https://a0.muscache.com/im/pictures/260fd609-7...,2217,https://www.airbnb.com/users/show/2217,Ian,2008-08-18,"Key Biscayne, Florida, United States",96%,within an hour,...,NaN,f,f,strict_14_with_grace_period,f,f,4,3.76,1,1
1,2695,Prenzlauer Berg close to Mauerpark,https://a0.muscache.com/im/pictures/97d4f6e8-1...,2986,https://www.airbnb.com/users/show/2986,Michael,2008-09-16,"Berlin, Berlin, Germany",NaN,NaN,...,NaN,f,f,flexible,f,f,1,1.42,1,1
2,3176,Fabulous Flat in great Location,https://a0.muscache.com/im/pictures/243355/84a...,3718,https://www.airbnb.com/users/show/3718,Britta,2008-10-19,"Coledale, New South Wales, Australia",100%,within a day,...,NaN,t,f,strict_14_with_grace_period,f,f,1,1.25,1,1
3,3309,BerlinSpot Schöneberg near KaDeWe,https://a0.muscache.com/im/pictures/29054294/b...,4108,https://www.airbnb.com/users/show/4108,Jana,2008-11-07,"Berlin, Berlin, Germany",NaN,NaN,...,NaN,f,f,strict_14_with_grace_period,f,f,1,0.39,1,1
4,7071,BrightRoom with sunny greenview!,https://a0.muscache.com/im/pictures/21278/32a1...,17391,https://www.airbnb.com/users/show/17391,Bright,2009-05-16,"Berlin, Berlin, Germany",100%,within an hour,...,NaN,f,f,moderate,f,f,1,1.75,1,1


In [107]:

for index, row in listings_summary.iterrows():
    if isnan(row["host_thumbnail_url"]):
        listings_summary["is_host_thumbnail_url"] = 1
    else:
        listings_summary["is_host_thumbnail_url"] = 0
listings_summary.head()

,id,name,picture_url,host_id,host_url,host_name,host_since,host_location,host_response_rate,host_response_time,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,is_host_picture_url,is_host_url,is_host_thumbnail_url
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,https://a0.muscache.com/im/pictures/260fd609-7...,2217,https://www.airbnb.com/users/show/2217,Ian,2008-08-18,"Key Biscayne, Florida, United States",96%,within an hour,...,f,f,strict_14_with_grace_period,f,f,4,3.76,1,1,1
1,2695,Prenzlauer Berg close to Mauerpark,https://a0.muscache.com/im/pictures/97d4f6e8-1...,2986,https://www.airbnb.com/users/show/2986,Michael,2008-09-16,"Berlin, Berlin, Germany",NaN,NaN,...,f,f,flexible,f,f,1,1.42,1,1,1
2,3176,Fabulous Flat in great Location,https://a0.muscache.com/im/pictures/243355/84a...,3718,https://www.airbnb.com/users/show/3718,Britta,2008-10-19,"Coledale, New South Wales, Australia",100%,within a day,...,t,f,strict_14_with_grace_period,f,f,1,1.25,1,1,1
3,3309,BerlinSpot Schöneberg near KaDeWe,https://a0.muscache.com/im/pictures/29054294/b...,4108,https://www.airbnb.com/users/show/4108,Jana,2008-11-07,"Berlin, Berlin, Germany",NaN,NaN,...,f,f,strict_14_with_grace_period,f,f,1,0.39,1,1,1
4,7071,BrightRoom with sunny greenview!,https://a0.muscache.com/im/pictures/21278/32a1...,17391,https://www.airbnb.com/users/show/17391,Bright,2009-05-16,"Berlin, Berlin, Germany",100%,within an hour,...,f,f,moderate,f,f,1,1.75,1,1,1


In [108]:
listings_summary = listings_summary.drop("host_thumbnail_url", axis = 1)
listings_summary = listings_summary.drop("host_url", axis = 1)
listings_summary = listings_summary.drop("host_picture_url", axis = 1)
listings_summary.head()

,id,name,picture_url,host_id,host_name,host_since,host_location,host_response_rate,host_response_time,host_acceptance_rate,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,is_host_picture_url,is_host_url,is_host_thumbnail_url
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,https://a0.muscache.com/im/pictures/260fd609-7...,2217,Ian,2008-08-18,"Key Biscayne, Florida, United States",96%,within an hour,NaN,...,f,f,strict_14_with_grace_period,f,f,4,3.76,1,1,1
1,2695,Prenzlauer Berg close to Mauerpark,https://a0.muscache.com/im/pictures/97d4f6e8-1...,2986,Michael,2008-09-16,"Berlin, Berlin, Germany",NaN,NaN,NaN,...,f,f,flexible,f,f,1,1.42,1,1,1
2,3176,Fabulous Flat in great Location,https://a0.muscache.com/im/pictures/243355/84a...,3718,Britta,2008-10-19,"Coledale, New South Wales, Australia",100%,within a day,NaN,...,t,f,strict_14_with_grace_period,f,f,1,1.25,1,1,1
3,3309,BerlinSpot Schöneberg near KaDeWe,https://a0.muscache.com/im/pictures/29054294/b...,4108,Jana,2008-11-07,"Berlin, Berlin, Germany",NaN,NaN,NaN,...,f,f,strict_14_with_grace_period,f,f,1,0.39,1,1,1
4,7071,BrightRoom with sunny greenview!,https://a0.muscache.com/im/pictures/21278/32a1...,17391,Bright,2009-05-16,"Berlin, Berlin, Germany",100%,within an hour,NaN,...,f,f,moderate,f,f,1,1.75,1,1,1


In [109]:
for index, row in listings_summary.iterrows():
    if isnan(row["picture_url"]):
        listings_summary["is_picture_url"] = 1
    else:
        listings_summary["is_picture_url"] = 0

listings_summary = listings_summary.drop("picture_url", axis = 1)

listings_summary.head()

,id,name,host_id,host_name,host_since,host_location,host_response_rate,host_response_time,host_acceptance_rate,host_is_superhost,...,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,is_host_picture_url,is_host_url,is_host_thumbnail_url,is_picture_url
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,2217,Ian,2008-08-18,"Key Biscayne, Florida, United States",96%,within an hour,NaN,t,...,f,strict_14_with_grace_period,f,f,4,3.76,1,1,1,1
1,2695,Prenzlauer Berg close to Mauerpark,2986,Michael,2008-09-16,"Berlin, Berlin, Germany",NaN,NaN,NaN,f,...,f,flexible,f,f,1,1.42,1,1,1,1
2,3176,Fabulous Flat in great Location,3718,Britta,2008-10-19,"Coledale, New South Wales, Australia",100%,within a day,NaN,f,...,f,strict_14_with_grace_period,f,f,1,1.25,1,1,1,1
3,3309,BerlinSpot Schöneberg near KaDeWe,4108,Jana,2008-11-07,"Berlin, Berlin, Germany",NaN,NaN,NaN,f,...,f,strict_14_with_grace_period,f,f,1,0.39,1,1,1,1
4,7071,BrightRoom with sunny greenview!,17391,Bright,2009-05-16,"Berlin, Berlin, Germany",100%,within an hour,NaN,t,...,f,moderate,f,f,1,1.75,1,1,1,1


In [111]:
import csv

# open the file in the write mode
with open('new_listings.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(listings_summary)

In [112]:
listings_summary.to_csv("new_listings.csv", encoding='utf-8', index=False)

In [121]:
len(listings_summary)

22552

In [122]:
listings_summary.columns

Index(['id', 'name', 'host_id', 'host_name', 'host_since', 'host_location',
       'host_response_rate', 'host_response_time', 'host_acceptance_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'latitude', 'longitude', 'is_location_exact',
       'property_type', 'accommodates', 'bedrooms', 'bathrooms', 'beds',
       'bed_type', 'square_feet', 'price', 'weekly_price', 'monthly_price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'calendar_updated',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'first_review', 'last_review',
       'review_score